# Parsing with TextFSM
A regular expression template based parser, wich return structured data from CLI output.  
https://github.com/google/textfsm


## Parsing a show command
Let's get some structured data from CLI output of the"show interface description" command.

This is an extract of what we get from the output :


`
Hu0/0/0/21/0       admin-down  admin-down
Te0/0/0/0          admin-down  admin-down
Te0/0/0/1          up          up          Connected with ANT - to AMALFI
Te0/0/0/2          up          up          Connected with ANT
Te0/0/0/3          up          up
Te0/0/0/4          admin-down  admin-down
Te0/0/0/5          up          up          Link to Bologna 0/1/0/5
Te0/0/0/6          admin-down  admin-down
Te0/0/0/7          up          up          to Honfleur tenG7
Te0/0/0/8          up          up
Te0/0/0/9          up          up
Te0/0/0/10         up          up          to Bologna int teng10
Te0/0/0/11         admin-down  admin-down
Te0/0/0/12         admin-down  admin-down
Te0/0/0/13         admin-down  admin-down
Te0/0/0/14         admin-down  admin-down
Te0/0/0/15         admin-down  admin-down
Te0/0/0/16         up          up          to int tenG16 Amalfi
Te0/0/0/17         admin-down  admin-down
Te0/0/0/18         admin-down  admin-down
Te0/0/0/19         admin-down  admin-down
Gi0/3/1/0          up          up
`

First step, we write the template and save it to show_interfaces_description.tmpl :

`Value Interface ((Gi|Te|Hu)\S+)`  
`Value Status (\S+)`  
`Value Protocol (\S+)`  
`Value Description (.+)`  
`Start`  
 
`^${Interface} \s+${Status} \s+${Protocol} \s+${Description} -> Record`  

The template is divided in two sections :
1. Values statements
2. States definitions

### 1. Values statements:
Values statements define the value we want to extract in the line.
If we look at first line the interface value, the regex ((Gi|Te|Hu)\S+) match only the lines which begin by Gi or Te or Hu, \S match any non-whitespace character, + is a quantifier put after \S will match one or more non whitespace.

Thus in the line :  
`Hu0/3/0/5          up          up          Link to Bologna 0/1/0/5`  
The regex **((Gi|Te|Hu)\S+)** match **Hu0/3/0/5**  


### 2. States definitions:
A state is defined by a regex list which will be tested against each line.  
In TextFSM a `Start` state is mandatory. 
In our example there is only one regex in only one state :   
`^${Interface} \s+${Status} \s+${Protocol} \s+${Description}`  
Followed by the directive `-> Record` will put the 4 values in a line of our list of results when there is a match.


More ressources on regular expression can be found :  
-   https://regexone.com/ interactive tutorial on regex.
-   https://regex101.com/ develop and debug your regex.

Some TextFSM ressources :
-   https://github.com/google/textfsm/wiki The TextFSM wiki on Github.
-   https://github.com/networktocode/ntc-templates A large collection of TextFSM based templates.



## Python code :
Let's parse the cli output of `show interface description` with our template :

In [ ]:
import os
from inc import textfsm

template_path = os.path.join("textfsm_templates","show_interfaces_description.tmpl")
template = open(template_path)

my_parser = textfsm.TextFSM(template)

output_cli_path = os.path.join("cli_txt","cli_output.txt")
output_cli = open(output_cli_path).read()

my_data = my_parser.ParseText(output_cli)

#Filter in list comprehension :
filtered_data = [row for row in my_data if "." not in row[0]]

for data in filtered_data:
    print(data)

## Excel output

We can save our data in a Excel file using the openpyxl library.


In [ ]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.active
headers = ["Interface","Status","Protocol","Description"]
ws.append(headers)
for data in filtered_data:
    ws.append(data)
wb.save("show-interfaces-description.xlsx")

<img src="img\show_interface_description.png" width="500">

Our data are now stored in "show-interfaces-description.xlsx"
We can apply some styles in the Excel file.  
Let's check if an interface is down and put the cell in red.

In [ ]:
from openpyxl.styles import PatternFill

color = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type = "solid")

for row in ws.iter_rows(ws.min_row,ws.max_row,2,2):
    for cell in row:
        if "down" in cell.value:
            cell.fill = color

wb.save("colored-show-interfaces-description.xlsx")

<img src="img\colored_show_interface_description.png" width="500">

Openpyxl have numerous options and features : https://openpyxl.readthedocs.io/en/stable/index.html 